## 통계로는 알 수 없는 문맥 정보


- 지금까지 BOW 기반의 방법을 이용해서 다양한 분류 기법을 수행했다.
- 하지만 여기에는 문제가 있다. 단어들이 쓰여진 순서에 따른 문맥 정보를 이용할 수 없다는 것이다.
- 기본적으로 BOW 방식은 단어들의 순서를 무시하고, 단어가 사용된 횟수를 기반으로 문서에 대한 벡터를 만든다.
- 이러한 문제점을 해결하기 위해 제시된 방법은 문서를 다어들의 통계적인 값으로 표현하지 않고,
- 있는 그래도 단어의 시퀀스로 표현해서 처리하는 것ㄷ이다.
- 딥러닝 기법은 이와 같은 요구에 의해 사용되기 시작했다.
- 딥러닝 기법 대신 BOW 방식을 그대로 쓰면서도 단어가 쓰여진 순서를 반영할 수 있는 방법이 N-gram이다.


## N-gram
- n-gram은 n개의 연속적인 단어들의 나열을 의미한다.
- 이는 하나의 토클이 두개 이상의 단어로 구성될 수 있는데 n이 2이면 bi-gram이라 부른다.
- 이 경우 하나의 토큰은 두 개의 단어로 구성된다.
- unigram: the, future, depends, on, what, we, do, in, the, present
- bi-gram: the future, future depends, depends on, on what...
- tri-gram: the future depends, future depends on, depends on what...


----
- n-gram의 본래 용도는 언어 모델의 특성에 있다.
- 언어 모델은 단어의 시퀀스에 대해 확률을 할당하는 모델을 말하는데, 이때 확률은 말뭉치에 나타난 단어 시퀀스의 빈도와 관련이 있다.
- 예를 들어 '배가 고파서 밥을 먹었다'와 '배가 고파서 밥을 치웠다' 중에서 앞 문장이 더 자연스럽다.
- 언어 모델은 이 두 문장에 확률을 부여하는데, 컴퓨터는 어떻게 확률을 부여할까.
- 확률을 계산하는데 참조하는 말뭉치에 앞 문장이 더 많이 나타나면 이를 기반으로 앞 문자에 더 높은 확률을 부여하면 된다.


----


- 문제는 '나는 배가 고파서 밥을 허겁지겁 먹었다'라는 문장은 나온 적이 없을때다.
- 이때 n-gram의 역할이 나온다.
- 말뭉치에 앞에서부터 이어지는 전체 문장이 없더라도 '밥을 허겁지겁'과 같은 bi-gram 말뭉치 어딘가에 난타났다면 이 분장에 대한 확률이 0 이상이 될 수 있다.
- 결국 bi-gram을 사용하면 적어도 두 단어로 이뤄진 시퀀스에 대해 파악하게 된다.
- 따라서 아주 제한적이지만 문맥에 대한 정보를 추가한다고 볼 수 있다.
- tri-gram을 사용하면 더 많은 정보를 얻을 수 있다.
- 하지만 n을 계속 늘려갈 수는 없다.
- 따라서 tri-gram 정도까지 쓰는 것이 알반적이다


In [31]:
from sklearn.datasets import fetch_20newsgroups


# 데이터셋 로드 (train 데이터를 기준으로)
# data = fetch_20newsgroups(subset='train')


# 데이터셋의 컬럼(속성) 확인
# print(data.keys())
# print(data.target_names)


# 20개의 토픽 중 선택하고자 하는 토픽을 리스트로 생성


# 20개의 토픽 중 4개의 토픽을 리스트로 생성
categories = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']


# 학습 데이터셋 로드
newsgroups_train = fetch_20newsgroups(subset='train',
# 메일 내용에서 hint가 되는 부분을 삭제 - 순수하게 내용만으로 분류
                                      remove=('headers', 'footers', 'quotes'), # 토픽의 정보가 있는 경우 제외
                                      categories=categories)


# 평가 데이터셋을 가져옴
newsgroups_test = fetch_20newsgroups(subset='test',
                                     remove=('headers', 'footers', 'quotes'),
                                     categories=categories)


print("훈련 세트: ", len(newsgroups_train.data))
print("테스트 세트: ", len(newsgroups_test.data))
print("선택 토픽: ", newsgroups_train.target_names)
print("훈련 라벨: ", set(newsgroups_train.target))


훈련 세트:  2034
테스트 세트:  1353
선택 토픽:  ['alt.atheism', 'comp.graphics', 'sci.space', 'talk.religion.misc']
훈련 라벨:  {0, 1, 2, 3}


In [32]:
# 카운트 기반 특성 추출
X_train = newsgroups_train.data
Y_train = newsgroups_train.target


X_test = newsgroups_test.data
Y_test = newsgroups_test.target


from sklearn.feature_extraction.text import TfidfVectorizer


tfidf = TfidfVectorizer(max_features=2000, min_df=5, max_df=0.5)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)


In [33]:
# n-gram을 이용한 문서 분류 구현
# unigram, bi-gram, tri-gram의 배교를 위해 Unigram으로 TfidfVectorizer 객체를 새로 생성하고
# 변환된 TF-IDF 벡터의 크기를 확인한다.
# n-gram에서 n이 바뀜에 따라 벡터의 크기가 어떻게 바뀌는지 보기 위해 max_features는 사용하지 않는다.
# 그 외에 토큰화를 위한 정규식을 인수로 주고 불용어사전을 이용한다.


import nltk

nltk.download("stopwords")


from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer


cachedStopWords = stopwords.words("english")


tfidf = TfidfVectorizer(
    token_pattern=r"(?u)\b\w\w+\b",  # 토큰화를 위한 정규식
    decode_error="ignore",
    lowercase=True,
    stop_words=stopwords.words("english"),  # 불용어사전
    max_df=0.5,
    min_df=2,
)


X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)


print(X_train_tfidf.shape)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\3호실-09\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


(2034, 12308)


In [34]:
# 릿지 회귀를 이용해 학습하고 성능 확인
# n-gram을 사용하면 변수가 늘어나고 이로 인해 과적합의 우려가 있기 때문에 릿지 회귀분석을 사용한다.
from sklearn.linear_model import RidgeClassifier

ridge_clf = RidgeClassifier()
ridge_clf.fit(X_train_tfidf, Y_train)
print("훈련 정확도: ", ridge_clf.score(X_train_tfidf, Y_train))
print("테스트 정확도: ", ridge_clf.score(X_test_tfidf, Y_test))


훈련 정확도:  0.976401179941003
테스트 정확도:  0.770879526977088


In [35]:
#bi-gram

# TfidfVectorizer의 ngram_range 파라미터 사용 
# 이 매개 변수는 () 안에 시작 n 값과 끝 n값으로 이뤄진 튜플이다 
# (1,2)듀플 쓰면 유니그램 바이그램 모두 씀 
# 바이그램은 (2,2)로 작성한다 

tfidf = TfidfVectorizer(
    token_pattern=r"(?u)\b\w\w+\b",  # 토큰화를 위한 정규식
    decode_error="ignore",
    lowercase=True,
    stop_words=stopwords.words("english"),  # 불용어사전
    max_df=0.5,
    min_df=2,
    ngram_range=(1,2)
)

X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)


print(X_train_tfidf.shape)

(2034, 28145)


In [36]:
# 바이그램이 어떤 방식으로 생성되는지 확인하기 위해 특성 이름을 출력한다. 
bigram_features = [f for f in tfidf.get_feature_names_out() if len(f.split()) > 1]
print(bigram_features[:10])

['00 10', '00 pm', '00 pounds', '00 thank', '000 000', '000 100', '000 bytes', '000 feet', '000 foot', '000 km']


In [37]:
# 바이그램으로 전환된 데이터를 통해 릿지 회귀 학습 
ridge_clf.fit(X_train_tfidf, Y_train)
print(
    "특성수를 늘린 후 릿지 훈련 세트 정확도 {:.3}".format(
        ridge_clf.score(X_train_tfidf, Y_train)
    )
)
# 테스트 세트 정확도
print(
    "특성수를 늘린 후 릿지 테스트 세트 정확도 {:.3}".format(
        ridge_clf.score(X_test_tfidf, Y_test)
    )
)

특성수를 늘린 후 릿지 훈련 세트 정확도 0.976
특성수를 늘린 후 릿지 테스트 세트 정확도 0.78


In [38]:
# 트라이 그램 사용 
tfidf = TfidfVectorizer(
    token_pattern=r"(?u)\b\w\w+\b",  # 토큰화를 위한 정규식
    decode_error="ignore",
    lowercase=True,
    stop_words=stopwords.words("english"),  # 불용어사전
    max_df=0.5,
    min_df=2,
    ngram_range=(1, 3)
)


X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)


print(X_train_tfidf.shape)

(2034, 34974)


In [39]:
trigram_features = [f for f in tfidf.get_feature_names_out() if len(f.split()) > 2]
print(trigram_features[:10])


['00 pounds us', '00 thank helping', '000 000 scale', '0000 155 1150', '00000 11888 86', '00041032 00000 11888', '0004422 293 4650', '0028 300 1200', '01 14 39', '01 v2 00']


In [40]:
ridge_clf.fit(X_train_tfidf, Y_train)

# 훈련 세트 정확도
print(
    "특성수를 늘린 후 릿지 훈련 세트 정확도 {:.3}".format(
        ridge_clf.score(X_train_tfidf, Y_train)
    )
)

# 테스트 세트 정확도
print(
    "특성수를 늘린 후 릿지 테스트 세트 정확도 {:.3}".format(
        ridge_clf.score(X_test_tfidf, Y_test)
    )
)


특성수를 늘린 후 릿지 훈련 세트 정확도 0.976
특성수를 늘린 후 릿지 테스트 세트 정확도 0.779
